In [1]:
%use dataframe
%use kandy
%use dataframe

In [27]:
import org.jetbrains.kotlinx.dataframe.DataFrame
import org.jetbrains.kotlinx.dataframe.api.*

class DataframeAnalyzer(private val dataframe: DataFrame<*>) {
    fun filter(filter: RowFilter<*>): DataframeAnalyzer {
        return DataframeAnalyzer(dataframe.filter(filter))
    }
    
    fun greaterThan(column: String, value: Double): DataframeAnalyzer {
        return filter { (it[column].toString().toDoubleOrNull() ?: return@filter false) > value }
    }
    
    fun lessThan(column: String, value: Double): DataframeAnalyzer {
        return filter { (it[column].toString().toDoubleOrNull() ?: return@filter false) < value }
    }
    
    fun doubleColumns(vararg columns: String): DataframeAnalyzer {
        return DataframeAnalyzer(dataframe.convert(*columns).with {
            it.toString().toDoubleOrNull()
        })
    }
    
    fun createPer90(column: String, ninetiesColumn: String = "Tempo de jogo - 90s"): DataframeAnalyzer {
        return DataframeAnalyzer(dataframe.insert(column + "/90") {
            (it[column].toString().toDoubleOrNull() ?: return@insert null) / (it[ninetiesColumn].toString().toDoubleOrNull() ?: error("ninetiesColumn is null"))
        }.after(column))
    }
    
    fun finish(): DataFrame<*> {
        return dataframe
    }
}

fun analyze(dataframe: DataFrame<*>): DataframeAnalyzer =
    DataframeAnalyzer(dataframe)

In [32]:
import org.jetbrains.kotlinx.kandy.ir.Plot

fun DataFrame<*>.footballPlot(
    x: String,
    y: String,
    min90s: Double = 0.0,
    per90: Boolean = false,
    ninetiesColumn: String = "Tempo de jogo - 90s"
): Plot {
    var analyzed = analyze(this)
        .doubleColumns(x, y)
        .greaterThan("90s", min90s)
    
    if (per90) {
        analyzed = analyzed.createPer90(x, ninetiesColumn).createPer90(y, ninetiesColumn)
    }
    
    return analyzed.finish().plot { 
        points {
            x(x + if (per90) "/90" else "")
            y(y + if (per90) "/90" else "")
            tooltips("Jogador")
        }
    }
}



In [36]:
val dataframe = DataFrame.readExcel("brasileirao.xlsx")

In [42]:
dataframe.footballPlot(
    x = "Progressão - PrgP",
    y = "Progressão - PrgC",
    min90s = 3.0,
    per90 = true
)

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="OdeJOy"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"Progressão - PrgC/90":[0.7317073170731708,1.0638297872340425,0.5263157894736842,0.0,2.439024390243903,3.0,0.2,0.0,4.0,2.4,1.8181818181818183,0.22727272727272727,0.0,0.5,0.75,0.25,0.0,0.625,0.45454545454545453,0.2,0.2173913043478261,0.5263157894736842,0.8333333333333334,1.6129032258064515,2.127659574468085,1.6,0.8,2.0,1.2244897959183672,0.9677419354838709,1.6,1.7647058823529411,0.0,0.0,1.1627906976744187,0.0,1.4893617021276595,2.307692307692308,0.5,1.5625,0.0,0.0,1.3513513513513513,1.0,1.5789473684210527,1.3157894736842106,0.0,0.7894736842105263,0.0,4.4186046511627906,2.4489795918367343,1.7857142857142858,1.75,1.081081081081081,0.0,0.0,0.5263157894736842,0.6,2.8125,1.1111111111111112,2.631578947368421,1.8,4.25,0.425531914893617,0.0,1.2820512820512822,2.75,4.4,0.6,1.8918918918918919,1.8918918918918919,0.5128205128205129,4.473684210526316,2.142857142857143,1.4,1.1764705882352942,1.1764705882352942,3.111111111111111,1.4,1.8,0.8620689655172414,2.5641025641025643,2.368421052631579,0.0,3.0,0.0,1.1363636363636362,3.75,2.368421052631579,4.468085106382978,0.0,0.30303030303030304,1.1111111111111112,0.2,0.0,1.75,1.1111111111111112,0.0,0.0,2.2727272727272725,2.4,0.7142857142857143,1.6216216216216215,3.7142857142857144,0.0,0.8333333333333333,1.25,1.25,0.6451612903225806,0.3125,2.954545454545454,0.6818181818181818,1.2903225806451613,0.25,0.16666666666666666,0.8108108108108107,0.20833333333333334,1.4705882352941178,0.5,2.75,4.285714285714286,0.3225806451612903,0.0,5.0,0.0,3.25,0.75,0.6451612903225806,2.6315789473684212],
"Progressão - PrgP/90":[7.8048780487804885,7.659574468085106,3.4210526315789473,0.0,6.097560975609756,3.0,2.8,0.0,2.6,4.4,5.151515151515151,4.545454545454545,0.0,1.75,1.0,1.25,3.2,5.0,0.9090909090909091,0.8,1.3043478260869565,3.68421052631579,3.3333333333333335,7.096774193548387,2.553191489361702,9.0,5.6,3.25,3.877551020408163,1.9354838709677418,3.0,4.705882352941177,0.2,0.0,3.255813953488372,0.0,10.0,1.794871794871795,2.75,11.5625,2.75,0.0,6.486486486486486,6.5,5.087719298245614,3.1578947368421053,3.5,3.4210526315789473,1.4583333333333335,3.255813953488372,4.693877551020408,5.0,3.25,2.4324324324324325,4.11764705882353,0.0,3.1578947368421053,4.6,2.8125,2.5,3.1578947368421053,4.6,5.25,1.702127659574468,1.8918918918918919,3.3333333333333335,5.75,2.0,3.0,2.7027027027027026,1.3513513513513513,4.871794871794872,1.842105263157895,4.523809523809524,6.2,3.823529411764706,2.058823529411765,2.4444444444444446,1.8,4.6,3.793103448275862,5.128205128205129,6.052631578947369,0.0,3.4,3.4285714285714284,1.5909090909090908,10.75,8.947368421052632,3.404255319148936,0.0,1.2121212121212122,2.7777777777777777,3.2,0.0,4.0,3.5555555555555554,0.0,0.0,2.727272727272727,6.2,2.619047619047619,5.405405405405405,8.285714285714286,0.0,3.888888888888889,3.75,3.125,2.258064516129032,5.3125,3.6363636363636362,1.3636363636363635,3.5483870967741935,1.0,3.5,2.4324324324324325,1.6666666666666667,4.705882352941177,6.0,5.0,1.1428571428571428,2.5806451612903225,0.0,3.5294117647058822,0.0,7.0,5.5,3.5483870967741935,3.68421052631579],
"Jogador":["Caio Alexandre","Alisson","Allan","Anderson","Guilherme Arana","Luiz Araújo","Robert Arboleda","Lucas Arcanjo","Jhon Arias","Santiago Arias","Lucas Barbosa","Bastos","Bento","Marllon Borges","Rafael Borré","Emanuel Brítez","Fabrício Bruno","Fabricio Bustos","Jonathan Calleri","Luan Cândido","Germán Cano","Juninho Capixaba","Murilo Cerqueira","Franco Cristaldo","David","Nicolás De La Cruz","Diego","Endrick","Erick","Erick","Lucas Esquivel","Lucas Esteves","Éverson","Fábio","Fagner","Marcos Felipe","Fernandinho","Luiz Fernando","Alan Franco","Marlon Fr